In [ ]:
#import tensorflow as tf
#import numpy as np
#import _pickle as pickle
#from lib.preprocessing import load_piece, rearrange_data
#from lib.model_assests import predictions_to_notes, create_midi

In [1]:
from __init__ import *

/usr/local/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sess = tf.Session()
new_saver = tf.train.import_meta_graph('../checkpoints/2/model-test-1.meta')
new_saver.restore(sess, tf.train.latest_checkpoint('../checkpoints/2/'))
graph = tf.get_default_graph()

INFO:tensorflow:Restoring parameters from ../checkpoints/2/model-test-1


In [3]:
config = Config()
config.num_prev = 20
config.batch_size = 14
config.num_steps = 10
config.element_size = 19*3
config.epochs = 100
config.num_layers = 1
config.print_step = 20
config.save_model = False
config.save_after_epoch = 1
config.model_save_path = '../checkpoints/2/model-test'


In [4]:
raw_data = load_piece('../data/Mozart_pickles/', 2)
raw_data = rearrange_data(raw_data, config.num_prev)
print('data loaded in shape',raw_data.shape)
data_obj = Input(raw_data, config)

data loaded in shape (5224, 20, 57)


In [5]:
logits = graph.get_tensor_by_name('Forward_propagation/logits:0')
X = graph.get_tensor_by_name('data_tensors/data_input:0')
init_state = graph.get_tensor_by_name('initial_state/init_state:0')
is_training = graph.get_tensor_by_name('training_flag:0')
output = graph.get_tensor_by_name('Forward_propagation/output:0')

In [6]:
current_state = np.zeros([2, config.batch_size, data_obj.hidden_size])

In [7]:
x_input, _ = data_obj.next_batch()
print (x_input.shape)

(14, 10, 1140)


In [8]:
[preds] = sess.run([logits], feed_dict={X:x_input, init_state:current_state, is_training:False})

In [9]:
for op in graph.get_operations():
    if 'outputx2' in op.name : op.name : print (op.name,'\t',op.type, op.values())

In [10]:
notes = predictions_to_notes(preds)

In [11]:
create_midi(notes, 'test.mid')

midi is generated and saved to disk


In [12]:
notes

['G5.',
 'G5.',
 'B5.',
 'B4.',
 'G4.',
 'G5.',
 'G4.',
 'B5.',
 'B4.',
 'G4.',
 'G5.',
 'G5.',
 'G5.',
 'D5.',
 'D4.',
 'G5.',
 'G5.',
 'B5.',
 'G5.',
 'B4.',
 'C5.',
 'G5.',
 'G4.',
 'G4.',
 'D4.',
 'G4.',
 'G4.',
 'G4.',
 'G4.',
 'B3.',
 'D5.',
 'G4.',
 'G4.',
 'G4.',
 'D5.',
 'G4.',
 'B4.',
 'B4.',
 'D4.',
 'D4.',
 'G4.',
 'D5.',
 'G4.',
 'D4.',
 'D4.',
 'G5.',
 'G4.',
 'G4.',
 'G4.',
 'G5.',
 'G5.',
 'D5.',
 'G4.',
 'G4.',
 'G4.',
 'G4.',
 'G4.',
 'G4.',
 'B4.',
 'D4.',
 'G5.',
 'G5.',
 'G4.',
 'D5.',
 'G5.',
 'G5.',
 'G5.',
 'B4.',
 'G4.',
 'B4.',
 'G5.',
 'G5.',
 'G4.',
 'D5.',
 'G4.',
 'B5.',
 'A4.',
 'D4.',
 'G4.',
 'B3.',
 'G5.',
 'G3.',
 'D4.',
 'D5.',
 'D4.',
 'D5.',
 'G5.',
 'C4.',
 'D5.',
 'G4.',
 'D5.',
 'G5.',
 'G4.',
 'D4.',
 'G5.',
 'B4.',
 'G4.',
 'G4.',
 'B5.',
 'B5.',
 'G5.',
 'D5.',
 'G4.',
 'G4.',
 'G4.',
 'D4.',
 'G4.',
 'G4.',
 'G4.',
 'B4.',
 'G5.',
 'G4.',
 'B4.',
 'G5.',
 'G4.',
 'B4.',
 'B5.',
 'G4.',
 'D4.',
 'G4.',
 'G4.',
 'G5.',
 'G5.',
 'D4.',
 'G4.',


In [14]:
note.Note('A#')

<music21.note.Note A#>

In [17]:
offset = 0
output_notes = []

for pattern in notes:
    # pattern is a chord
    notes_in_pattern = pattern.split('.')[:-1]
    if len(notes_in_pattern) > 1 : # is a chord
        notes = []
        for current_note in notes_in_pattern:
            new_note = note.Note(current_note)
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a single note
    #else :
    #    new_note = note.Note(notes_in_pattern[0])
    #    new_note.storedInstrument = instrument.Piano()
    #    new_note.offset = offset
    #    output_notes.append(new_note)
    #increase offset each step
    offset += .5


AccidentalException: #- is not a supported accidental type

In [18]:
current_note

'F#-10'